In [1]:
import torch
torch.cuda.empty_cache()

In [2]:
!pip install openai transformers datasets rouge_score nltk

In [3]:
from datasets import load_dataset
from transformers import pipeline, AutoTokenizer
from sklearn.metrics import precision_score, recall_score, f1_score
from rouge_score import rouge_scorer
import re
import json


In [4]:
# # Load dataset in exctractive mode
# from datasets import load_dataset
# dataset = load_dataset("sobamchan/aclsum", "extractive")["test"]
# print(dataset[0].keys())

# model_name = "Qwen/Qwen3-4B"

# # Replace model name for Qwen or LLaMA
# summarizer = pipeline("text-generation", model=model_name)

# # replace tokenizer as needed as well
# tokenizer = AutoTokenizer.from_pretrained(model_name)

# gen = pipeline("text-generation", model=model_name, tokenizer=tokenizer, device_map="auto")

In [5]:
# Load dataset in extractive mode
from datasets import load_dataset
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

# Load the dataset
dataset = load_dataset("sobamchan/aclsum", "extractive")["test"]
print(dataset[0].keys())

# Define the model name
model_name = "Qwen/Qwen3-4B" # Use the chat-tuned model

# Load the tokenizer and model with trust_remote_code
# This is the correct and most reliable way
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True, device_map="auto")

# You can now create a single, correct pipeline if you want to use it
# or use the model.generate() method directly as we discussed.
summarizer = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto"
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


dict_keys(['id', 'source_sentences', 'challenge_sentences', 'approach_sentences', 'outcome_sentences', 'challenge_labels', 'approach_labels', 'outcome_labels'])


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0


In [6]:
print(dataset)

Dataset({
    features: ['id', 'source_sentences', 'challenge_sentences', 'approach_sentences', 'outcome_sentences', 'challenge_labels', 'approach_labels', 'outcome_labels'],
    num_rows: 100
})


In [7]:
print(dataset['challenge_labels'][0])

[1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


### preprocess into sentence label pair

This step is necessary as it will allow us to match the sentence itself with the label. This means that for every possible label (challenge, approach, outcome) we will store the sentences with their proper label in order to be able to evaluate the outcome accordingly.

This means that every phrase in the source_sentences section of the dataset will contain a label as well as the aspect. So, the first sentence might be assigned label 1 for aspect challenge and label 0 for aspects outcome and approach.

In [8]:
def prepare_aspect_data(example, aspect):
    return [
        {"sentence": sent, "label": lab, "aspect": aspect}
        for sent, lab in zip(example["source_sentences"], example[f"{aspect}_labels"])
    ]

sample = prepare_aspect_data(dataset[0], "challenge")
print(sample[:3])

[{'sentence': 'Handling terminology is an important matter in a translation workflow .', 'label': 1, 'aspect': 'challenge'}, {'sentence': 'However , current Machine Translation ( MT ) systems do not yet propose anything proactive upon tools which assist in managing terminological databases .', 'label': 1, 'aspect': 'challenge'}, {'sentence': 'In this work , we investigate several enhancements to analogical learning and test our implementation on translating medical terms .', 'label': 0, 'aspect': 'challenge'}]


In [9]:
sample_1 = prepare_aspect_data(dataset[0], "outcome")
print(sample_1[:3])

[{'sentence': 'Handling terminology is an important matter in a translation workflow .', 'label': 0, 'aspect': 'outcome'}, {'sentence': 'However , current Machine Translation ( MT ) systems do not yet propose anything proactive upon tools which assist in managing terminological databases .', 'label': 0, 'aspect': 'outcome'}, {'sentence': 'In this work , we investigate several enhancements to analogical learning and test our implementation on translating medical terms .', 'label': 0, 'aspect': 'outcome'}]


In [10]:
sample_2 = prepare_aspect_data(dataset[0], "approach")
print(sample_2[:3])

[{'sentence': 'Handling terminology is an important matter in a translation workflow .', 'label': 0, 'aspect': 'approach'}, {'sentence': 'However , current Machine Translation ( MT ) systems do not yet propose anything proactive upon tools which assist in managing terminological databases .', 'label': 0, 'aspect': 'approach'}, {'sentence': 'In this work , we investigate several enhancements to analogical learning and test our implementation on translating medical terms .', 'label': 1, 'aspect': 'approach'}]


To apply this to the whole dataset we can use the following code snippet

In [11]:
def add_triplets(example):
    aspects = ["challenge", "approach", "outcome"]
    # collect labels for each aspect
    aspect_labels = {a: [lab for _, lab in zip(example["source_sentences"], example[f"{a}_labels"])]
                     for a in aspects}
    # combine into triplets
    triplets = []
    for i, sent in enumerate(example["source_sentences"]):
        triplet = [aspect_labels["challenge"][i],
                   aspect_labels["approach"][i],
                   aspect_labels["outcome"][i]]
        triplets.append(triplet)
    example["triplets"] = triplets
    return example

labeled_dataset = dataset.map(add_triplets)

# quick peek
print(labeled_dataset[0]["source_sentences"][:10])
print(labeled_dataset[0]["triplets"][:10])


['Handling terminology is an important matter in a translation workflow .', 'However , current Machine Translation ( MT ) systems do not yet propose anything proactive upon tools which assist in managing terminological databases .', 'In this work , we investigate several enhancements to analogical learning and test our implementation on translating medical terms .', 'We show that the analogical engine works equally well when translating from and into a morphologically rich language , or when dealing with language pairs written in different scripts .', 'Combining it with a phrasebased statistical engine leads to significant improvements .', 'If machine translation is to meet commercial needs , it must offer a sensible approach to translating terms .', 'Currently , MT systems offer at best database management tools which allow a human ( typically a translator , a terminologist or even the vendor of the system ) to specify bilingual terminological entries .', 'More advanced tools are mean

In [12]:
from datasets import load_dataset

# Function: expand one document into list of sentence dicts
def expand_doc(example):
    return {
        "sentences": [
            {
                "sentence": sent,
                "label_ch": int(ch),
                "label_ap": int(ap),
                "label_oc": int(oc),
            }
            for sent, ch, ap, oc in zip(
                example["source_sentences"],
                example["challenge_labels"],
                example["approach_labels"],
                example["outcome_labels"]
            )
        ]
    }

expanded = dataset.map(expand_doc)

print(expanded[1]["sentences"][:3])
print(f'check length: {len(expanded[1]["sentences"])} {len(dataset[1]["source_sentences"])}')


[{'label_ap': 0, 'label_ch': 1, 'label_oc': 0, 'sentence': 'Reasoning about implied relationships ( e.g. paraphrastic , common sense , encyclopedic ) between pairs of words is crucial for many cross-sentence inference problems .'}, {'label_ap': 1, 'label_ch': 0, 'label_oc': 0, 'sentence': 'This paper proposes new methods for learning and using embeddings of word pairs that implicitly represent background knowledge about such relationships .'}, {'label_ap': 1, 'label_ch': 0, 'label_oc': 0, 'sentence': 'Our pairwise embeddings are computed as a compositional function on word representations , which is learned by maximizing the pointwise mutual information ( PMI ) with the contexts in which the two words cooccur .'}]
check length: 32 32


## prompting techniques

the vanilla prompt has only the task to select the most important sentences regardless of the aspect

In [13]:
def simple_vanilla_prompt(sentences):
    """
    Generates a simple, general-purpose vanilla prompt for extractive summarization.
    """
    numbered = [f"Sentence {i+1}: {s}" for i, s in enumerate(sentences)]
    input_text = "\n".join(numbered)

    return f'''You are an expert in extractive summarization. Your task is to select the most important sentences from the document.

Input:
{input_text}

Rules:
- Select ONLY the most important sentences.
- If no sentences are important, return an empty list.
- Indices are 1-based.
- Return ONLY valid JSON.

Return format:
{{"selected_sentences": [list_of_sentence_numbers]}}'''

this second vanilla prompt instead selects the most important phrases based on the aspect, so which are the most important phrases connected to challenge, approach and outcome

In [14]:
# vanilla prompt
# needs to be called three times on the same phrase to understand the three aspects
def vanilla_prompt(sentences, target_label):
    # target_label ∈ {"challenge","approach","outcome"}
    numbered = [f"Sentence {i+1}: {s}" for i, s in enumerate(sentences)]
    input_text = "\n".join(numbered)
    return f'''You are an expert in extractive summarization. Your task is to select sentences that express the "{target_label}" aspect of the document.

Aspect definitions:
- challenge: problem, gap, limitation, unmet need, difficulty/motivation.
- approach: method, model, system, algorithm, dataset design, procedure.
- outcome: results, findings, improvements, metrics, performance, impact.

Input:
{input_text}

Rules:
- Select ONLY sentences that primarily express the "{target_label}" aspect.
- If none match, return an empty list.
- Indices are 1-based.
- Return ONLY valid JSON.

Return format:
{{"selected_sentences": [list_of_sentence_numbers]}}'''


**Least to most** prompting technique implemented both in its aspect-based version and in the simple version. Ask the model to identify the overall purpose of the document before returning either the aspect-based sentences or the overall most important sentences.

In [15]:
def least_to_most_prompt(sentences, target_label):
    numbered = [f"Sentence {i+1}: {s}" for i, s in enumerate(sentences)]
    input_text = "\n".join(numbered)

    return f'''You are an expert in extractive summarization. Your task is to select sentences that express the "{target_label}" aspect of the document.

Aspect definitions:
- challenge: problem, gap, limitation, unmet need, difficulty/motivation.
- approach: method, model, system, algorithm, dataset design, procedure.
- outcome: results, findings, improvements, metrics, performance, impact.

Input:
{input_text}

First, consider the overall purpose of the document and how each sentence contributes to it.
Then, from that understanding, select ONLY sentences that primarily express the "{target_label}" aspect.
If none match, return an empty list.
Indices are 1-based.
Return ONLY valid JSON.

Return format:
{{"selected_sentences": [list_of_sentence_numbers]}}'''

In [16]:
def least_to_most_simple_prompt(sentences):
    numbered = [f"Sentence {i+1}: {s}" for i, s in enumerate(sentences)]
    input_text = "\n".join(numbered)

    return f'''You are an expert in extractive summarization. Your task is to select the most important sentences from the document.

Input:
{input_text}

First, identify the main topics and key arguments of the document.
Then, select ONLY the sentences that directly relate to those topics.
If no sentences are important, return an empty list.
Indices are 1-based.
Return ONLY valid JSON.

Return format:
{{"selected_sentences": [list_of_sentence_numbers]}}'''

**Tool-augmented prompting**. Here the model is instructed to use a "tool" or internal function to aid its reasoning. The tool is not a real external program but a conceptual instruction within the prompt itself.



In [17]:
def tool_augmented_prompt(sentences, target_label):
    numbered = [f"Sentence {i+1}: {s}" for i, s in enumerate(sentences)]
    input_text = "\n".join(numbered)

    return f'''You are an expert in extractive summarization. Your task is to select sentences that express the "{target_label}" aspect of the document.

Aspect definitions:
- challenge: problem, gap, limitation, unmet need, difficulty/motivation.
- approach: method, model, system, algorithm, dataset design, procedure.
- outcome: results, findings, improvements, metrics, performance, impact.

Input:
{input_text}

Instructions:
1. For each sentence, use the internal `check_aspect(sentence, aspect)` tool.
2. The tool's output is 'match' if the sentence primarily describes the "{target_label}" aspect, otherwise it is 'no_match'.
3. List the sentences that result in a 'match'.
4. If no sentences match, return an empty list.
5. Indices are 1-based.
6. Return ONLY valid JSON.

Return format:
{{"selected_sentences": [list_of_sentence_numbers]}}'''

In [18]:
def tool_augmented_simple_prompt(sentences):
    numbered = [f"Sentence {i+1}: {s}" for i, s in enumerate(sentences)]
    input_text = "\n".join(numbered)

    return f'''You are an expert in extractive summarization. Your task is to select the most important sentences from the document.

Input:
{input_text}

Instructions:
1. For each sentence, use the internal `check_importance(sentence)` tool.
2. The tool's output is 'important' if the sentence is central to the main idea, otherwise it is 'not_important'.
3. List the sentences that result in an 'important' output.
4. If no sentences are important, return an empty list.
5. Indices are 1-based.
6. Return ONLY valid JSON.

Return format:
{{"selected_sentences": [list_of_sentence_numbers]}}'''

**scoring-based prompting** instead gives a score to each sentence based on the relevance to the task. Also here we have the aspect-based version and the simple, importance-based one.

In [19]:
def scoring_based_prompt(sentences, target_label):
    numbered = [f"Sentence {i+1}: {s}" for i, s in enumerate(sentences)]
    input_text = "\n".join(numbered)

    return f'''You are an expert in extractive summarization. Your task is to select sentences that express the "{target_label}" aspect of the document.

Aspect definitions:
- challenge: problem, gap, limitation, unmet need, difficulty/motivation.
- approach: method, model, system, algorithm, dataset design, procedure.
- outcome: results, findings, improvements, metrics, performance, impact.

Input:
{input_text}

Instructions:
1. For each sentence, assign a score from 1 (low relevance) to 5 (high relevance) for how well it expresses the "{target_label}" aspect.
2. Only select sentences with a score of 4 or 5.
3. If no sentences meet the threshold, return an empty list.
4. Indices are 1-based.
5. Return ONLY valid JSON.

Return format:
{{"selected_sentences": [list_of_sentence_numbers]}}'''

In [20]:
def scoring_based_simple_prompt(sentences):
    numbered = [f"Sentence {i+1}: {s}" for i, s in enumerate(sentences)]
    input_text = "\n".join(numbered)

    return f'''You are an expert in extractive summarization. Your task is to select the most important sentences from the document.

Input:
{input_text}

Instructions:
1. For each sentence, assign a score from 1 (low importance) to 5 (high importance) for how central it is to the document's main idea.
2. Only select sentences with a score of 4 or 5.
3. If no sentences meet the threshold, return an empty list.
4. Indices are 1-based.
5. Return ONLY valid JSON.

Return format:
{{"selected_sentences": [list_of_sentence_numbers]}}'''

**self-ask prompting** involves the model reasoning through a series of yes-no questions and using the answers to reach a conclustion.

In [21]:
def self_ask_prompt(sentences, target_label):
    numbered = [f"Sentence {i+1}: {s}" for i, s in enumerate(sentences)]
    input_text = "\n".join(numbered)

    return f'''You are an expert in extractive summarization. Your task is to select sentences that express the "{target_label}" aspect of the document.

Aspect definitions:
- challenge: problem, gap, limitation, unmet need, difficulty/motivation.
- approach: method, model, system, algorithm, dataset design, procedure.
- outcome: results, findings, improvements, metrics, performance, impact.

Input:
{input_text}

Instructions:
1. Reason step-by-step. For each sentence, ask the question: "Does this sentence primarily express the "{target_label}" aspect?"
2. Answer the question with "Yes" or "No".
3. Compile a list of all sentences for which the answer was "Yes".
4. If no sentences meet the criteria, return an empty list.
5. Indices are 1-based.
6. Return ONLY valid JSON.

Return format:
{{"selected_sentences": [list_of_sentence_numbers]}}'''

In [22]:
def self_ask_simple_prompt(sentences):
    numbered = [f"Sentence {i+1}: {s}" for i, s in enumerate(sentences)]
    input_text = "\n".join(numbered)

    return f'''You are an expert in extractive summarization. Your task is to select the most important sentences from the document.

Input:
{input_text}

Instructions:
1. Reason step-by-step. First, ask the question: "What is the main idea of this document?"
2. Then, for each sentence, ask: "Does this sentence support the main idea?"
3. Compile a list of all sentences for which the answer was "Yes".
4. If no sentences are important, return an empty list.
5. Indices are 1-based.
6. Return ONLY valid JSON.

Return format:
{{"selected_sentences": [list_of_sentence_numbers]}}'''

- Transform gold standard to match output for each label:

  so for example if the challenge_labels looks like this:

  `[1,0,0,1,1,0]`

  it will become

  `[1, 4, 5]`



In [23]:
# I want also the gold standard to match the same output
ASPECTS = ["challenge", "approach", "outcome"]

def labels_to_indices(labels):
    """0/1 list -> 1-based indices of 1s"""
    return [i+1 for i, v in enumerate(labels) if int(v) == 1]

def gold_for_doc(example):
    """
    Build gold indices per aspect for ONE document.
    Returns: {"challenge":[...], "approach":[...], "outcome":[...]}
    """
    return {
        "challenge": labels_to_indices(example["challenge_labels"]),
        "approach":  labels_to_indices(example["approach_labels"]),
        "outcome":   labels_to_indices(example["outcome_labels"]),
    }

def gold_for_dataset(ds):
    """
    Build gold indices per aspect for ALL docs.
    Returns a list aligned with ds, where item i is gold_for_doc(ds[i])
    """
    return [gold_for_doc(ex) for ex in ds]

gold_all = gold_for_dataset(dataset)
print(gold_all[0])


{'challenge': [1, 2, 7, 11], 'approach': [3, 15, 19, 26, 27], 'outcome': [4, 5, 20, 21, 30, 31]}


-

In [24]:
# --- JSON parsing that tolerates extra text ---
def safe_extract_json(text: str):
    text = text.strip()
    # try direct
    try:
        return json.loads(text)
    except Exception:
        pass
    # try to find {...}
    m = re.search(r"\{.*\}", text, flags=re.S)
    if m:
        try:
            return json.loads(m.group(0))
        except Exception:
            return {}
    return {}

In [25]:
import json, re
from transformers import AutoTokenizer, pipeline

In [26]:
import json, re

def safe_extract_json_strict(text: str):
    """
    Strictly extract the model's JSON:
      - Prefer a JSON object that contains "selected_sentences".
      - Else accept a SINGLE standalone bracketed list on its own line.
      - Otherwise return {} (no guesses; avoids capturing numbers from the prompt).
    """
    text = text.strip()

    try:
        js = json.loads(text)
        if isinstance(js, dict) and "selected_sentences" in js:
            return js
        if isinstance(js, list):
            return {"selected_sentences": js}
    except Exception:
        pass

    text_clean = re.sub(r"^```[\w-]*\s*\n", "", text, flags=re.S)
    text_clean = re.sub(r"\n```$", "", text_clean, flags=re.S).strip()

    objs = re.findall(r"\{[\s\S]*?\}", text_clean)
    for s in reversed(objs):
        try:
            js = json.loads(s)
            if isinstance(js, dict) and "selected_sentences" in js:
                return js
        except Exception:
            continue

    m = re.search(r"(?m)^\s*\[(?:\s*\d+\s*(?:,\s*\d+\s*)*)?\]\s*$", text_clean)
    if m:
        try:
            arr = json.loads(m.group(0))
            return {"selected_sentences": arr}
        except Exception:
            pass

    return {}


- The qwen model is pretrained on chat-like inputs, and therefore the input we have needs to be translated as a chat as well

In [27]:
def predict_indices_for_aspect(prompt_technique, sentences, target_label, max_new_tokens=256, show_raw=False):
    # Build prompt and format as chat text
    user_prompt = prompt_technique(sentences, target_label)
    chat_text = tokenizer.apply_chat_template(
        [
            {"role": "system", "content": "You are an expert in extractive summarization."},
            {"role": "user", "content": user_prompt}
        ],
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False
    )

    print(f'lenght= {len(tokenizer.encode(chat_text))}')

    if tokenizer.pad_token_id is None:
        tokenizer.pad_token = tokenizer.eos_token

    # Use the model's generate method directly for robust generation
    inputs = tokenizer(chat_text, return_tensors='pt', padding=True)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=False,  # This is the only generation flag you need
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id
    )

    # Decode the generated tokens to a string
    out = tokenizer.decode(outputs[0], skip_special_tokens=True)

    if show_raw:
        print(f"\n[RAW OUTPUT for {target_label}]\n{out}\n")

    # Use the robust JSON parser on the decoded string
    js = safe_extract_json_strict(out)
    idxs = js.get("selected_sentences", [])

    # Sanitize indices
    n = len(sentences)
    cleaned = []
    for v in idxs:
        if isinstance(v, (int, float)):
            v = int(v)
            if 1 <= v <= n:
                cleaned.append(v)
    return sorted(set(cleaned))


In [28]:
def calculate_metrics(gold_for_doc, predicted_indices):

    # Convert lists to sets for efficient intersection and difference operations
    gold_set = set(gold_for_doc)
    predicted_set = set(predicted_indices)

    # Calculate True Positives (TP), False Positives (FP), and False Negatives (FN)
    true_positives = len(gold_set.intersection(predicted_set))
    false_positives = len(predicted_set.difference(gold_set))
    false_negatives = len(gold_set.difference(predicted_set))

    # Calculate Precision
    if true_positives + false_positives == 0:
        precision = 0.0
    else:
        precision = true_positives / (true_positives + false_positives)

    # Calculate Recall
    if true_positives + false_negatives == 0:
        recall = 0.0
    else:
        recall = true_positives / (true_positives + false_negatives)

    # Calculate F1-score
    if precision + recall == 0:
        f1_score = 0.0
    else:
        f1_score = 2 * (precision * recall) / (precision + recall)

    return {
        'precision': precision,
        'recall': recall,
        'f1_score': f1_score
    }

Evaluation loop:

The following function evaluates precision, recall and f1 for each aspect both for a single document and in an aggregate way

In [29]:
def run_evaluation(dataset, prompt_technique, gold_all, aspects, start=0, stop=None):
    if stop is None:
        stop = len(dataset)
    stop = min(stop, len(dataset))

    # Dictionaries to store metrics for macro-averaging
    all_metrics = {aspect: {"precision": [], "recall": [], "f1_score": []} for aspect in aspects}

    for doc_idx in range(start, stop):
        ex = dataset[doc_idx]
        sentences = ex["source_sentences"]
        gold_standard = gold_all[doc_idx] # given that gold_all = gold_for_dataset(dataset)

        print("="*80)
        print(f"Doc {doc_idx} (id={ex.get('id', 'NA')}) | #sentences={len(sentences)}")
        print("-"*80)

        for aspect in aspects:
            # get predicted indices for the current aspect
            predicted_indices = predict_indices_for_aspect(
                prompt_technique=prompt_technique,
                sentences=sentences,
                target_label=aspect,
            )

            # get gold standard for each aspect
            gold_indices = gold_standard.get(aspect, [])

            # 3. Calculate metrics
            metrics = calculate_metrics(gold_indices, predicted_indices)

            # Print per-document results
            print(f"  -> {aspect.capitalize():9s}")
            print(f"     Predicted: {sorted(predicted_indices)}")
            print(f"     Gold:      {sorted(gold_indices)}")
            print(f"     Metrics: P={metrics['precision']:.2f}, R={metrics['recall']:.2f}, F1={metrics['f1_score']:.2f}")
            print()

            # 4. Store metrics for aggregation
            all_metrics[aspect]["precision"].append(metrics['precision'])
            all_metrics[aspect]["recall"].append(metrics['recall'])
            all_metrics[aspect]["f1_score"].append(metrics['f1_score'])

    # After the loop, calculate and print the final aggregate scores
    print("\n" + "="*80)
    print("--- Final Aggregate Metrics ---")
    print("="*80)

    for aspect in aspects:
        avg_p = sum(all_metrics[aspect]["precision"]) / len(all_metrics[aspect]["precision"]) if all_metrics[aspect]["precision"] else 0
        avg_r = sum(all_metrics[aspect]["recall"]) / len(all_metrics[aspect]["recall"]) if all_metrics[aspect]["recall"] else 0
        avg_f1 = sum(all_metrics[aspect]["f1_score"]) / len(all_metrics[aspect]["f1_score"]) if all_metrics[aspect]["f1_score"] else 0

        print(f"  -> {aspect.capitalize()} Average:")
        print(f"     Precision: {avg_p:.2f}")
        print(f"     Recall:    {avg_r:.2f}")
        print(f"     F1-score:  {avg_f1:.2f}")
        print()

    print("="*80)

### Predictions for the first three documents for the **vanilla prompt**

In [31]:
run_evaluation(dataset, vanilla_prompt, gold_all, ASPECTS, stop=3)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Doc 0 (id=E09-1056) | #sentences=44
--------------------------------------------------------------------------------
lenght= 1551


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


  -> Challenge
     Predicted: [2, 6, 11, 13, 14, 37]
     Gold:      [1, 2, 7, 11]
     Metrics: P=0.33, R=0.50, F1=0.40

lenght= 1553


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


...

  -> Outcome  
     Predicted: [5, 6, 20, 21, 22, 26]
     Gold:      [5, 6, 13, 20, 21, 22, 26]
     Metrics: P=1.00, R=0.86, F1=0.92

Doc 2 (id=P01-1040) | #sentences=17
--------------------------------------------------------------------------------
lenght= 982


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


  -> Challenge
     Predicted: [1, 5, 6, 16]
     Gold:      [1, 5, 6]
     Metrics: P=0.75, R=1.00, F1=0.86

lenght= 984


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


  -> Approach 
     Predicted: [2, 7, 8, 12, 14]
     Gold:      [2, 7, 14]
     Metrics: P=0.60, R=1.00, F1=0.75

lenght= 982
  -> Outcome  
     Predicted: [9, 12, 17]
     Gold:      [3, 4, 12, 13]
     Metrics: P=0.33, R=0.25, F1=0.29


--- Final Aggregate Metrics ---
  -> Challenge Average:
     Precision: 0.56
     Recall:    0.83
     F1-score:  0.67

  -> Approach Average:
     Precision: 0.51
     Recall:    0.93
     F1-score:  0.64

  -> Outcome Average:
     Precision: 0.58
     Recall:    0.65
     F1-score:  0.59



### Predictions for the first three documents for the **least to most prompt**

In [32]:
# least_to_most_prompt
run_evaluation(dataset, least_to_most_prompt, gold_all, ASPECTS, stop=3)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Doc 0 (id=E09-1056) | #sentences=44
--------------------------------------------------------------------------------
lenght= 1568


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


  -> Challenge
     Predicted: [2, 6, 11, 13, 14, 37]
     Gold:      [1, 2, 7, 11]
     Metrics: P=0.33, R=0.50, F1=0.40

lenght= 1570


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


...

  -> Outcome  
     Predicted: [5, 6, 20, 21, 22, 26]
     Gold:      [5, 6, 13, 20, 21, 22, 26]
     Metrics: P=1.00, R=0.86, F1=0.92

Doc 2 (id=P01-1040) | #sentences=17
--------------------------------------------------------------------------------
lenght= 999


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


  -> Challenge
     Predicted: [1, 5, 6, 16]
     Gold:      [1, 5, 6]
     Metrics: P=0.75, R=1.00, F1=0.86

lenght= 1001


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


  -> Approach 
     Predicted: [2, 7, 8, 12, 14]
     Gold:      [2, 7, 14]
     Metrics: P=0.60, R=1.00, F1=0.75

lenght= 999
  -> Outcome  
     Predicted: [9, 12, 13, 17]
     Gold:      [3, 4, 12, 13]
     Metrics: P=0.50, R=0.50, F1=0.50


--- Final Aggregate Metrics ---
  -> Challenge Average:
     Precision: 0.56
     Recall:    0.83
     F1-score:  0.67

  -> Approach Average:
     Precision: 0.56
     Recall:    0.93
     F1-score:  0.69

  -> Outcome Average:
     Precision: 0.64
     Recall:    0.73
     F1-score:  0.66



### Predictions for the first three documents for the **tool augmented prompt**

In [33]:
# tool_augmented_prompt
run_evaluation(dataset, tool_augmented_prompt, gold_all, ASPECTS, stop=3)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Doc 0 (id=E09-1056) | #sentences=44
--------------------------------------------------------------------------------
lenght= 1600


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


  -> Challenge
     Predicted: [2, 6, 11, 13, 14, 37]
     Gold:      [1, 2, 7, 11]
     Metrics: P=0.33, R=0.50, F1=0.40

lenght= 1602


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


...

  -> Outcome  
     Predicted: [5, 6, 20, 21, 22, 26]
     Gold:      [5, 6, 13, 20, 21, 22, 26]
     Metrics: P=1.00, R=0.86, F1=0.92

Doc 2 (id=P01-1040) | #sentences=17
--------------------------------------------------------------------------------
lenght= 1031


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


  -> Challenge
     Predicted: [1, 5, 6]
     Gold:      [1, 5, 6]
     Metrics: P=1.00, R=1.00, F1=1.00

lenght= 1033


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


  -> Approach 
     Predicted: [2, 7, 8, 12, 14]
     Gold:      [2, 7, 14]
     Metrics: P=0.60, R=1.00, F1=0.75

lenght= 1031
  -> Outcome  
     Predicted: [9, 12, 17]
     Gold:      [3, 4, 12, 13]
     Metrics: P=0.33, R=0.25, F1=0.29


--- Final Aggregate Metrics ---
  -> Challenge Average:
     Precision: 0.64
     Recall:    0.83
     F1-score:  0.72

  -> Approach Average:
     Precision: 0.51
     Recall:    0.93
     F1-score:  0.64

  -> Outcome Average:
     Precision: 0.56
     Recall:    0.54
     F1-score:  0.54



### Predictions for the first three documents for the **scoring based prompt**

In [34]:
# scoring_based_prompt
run_evaluation(dataset, scoring_based_prompt, gold_all, ASPECTS, stop=3)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Doc 0 (id=E09-1056) | #sentences=44
--------------------------------------------------------------------------------
lenght= 1593


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


  -> Challenge
     Predicted: [2, 6, 11, 13, 14, 37]
     Gold:      [1, 2, 7, 11]
     Metrics: P=0.33, R=0.50, F1=0.40

lenght= 1595


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


...

  -> Outcome  
     Predicted: [5, 6, 20, 21, 22, 26]
     Gold:      [5, 6, 13, 20, 21, 22, 26]
     Metrics: P=1.00, R=0.86, F1=0.92

Doc 2 (id=P01-1040) | #sentences=17
--------------------------------------------------------------------------------
lenght= 1024


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


  -> Challenge
     Predicted: [1, 5, 6, 7]
     Gold:      [1, 5, 6]
     Metrics: P=0.75, R=1.00, F1=0.86

lenght= 1026


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


  -> Approach 
     Predicted: [2, 7, 8, 12, 14]
     Gold:      [2, 7, 14]
     Metrics: P=0.60, R=1.00, F1=0.75

lenght= 1024
  -> Outcome  
     Predicted: [9, 12, 17]
     Gold:      [3, 4, 12, 13]
     Metrics: P=0.33, R=0.25, F1=0.29


--- Final Aggregate Metrics ---
  -> Challenge Average:
     Precision: 0.56
     Recall:    0.83
     F1-score:  0.67

  -> Approach Average:
     Precision: 0.51
     Recall:    0.93
     F1-score:  0.64

  -> Outcome Average:
     Precision: 0.58
     Recall:    0.65
     F1-score:  0.59



### Predictions for the first three documents for the **self ask prompt**

In [35]:
# self_ask_prompt
run_evaluation(dataset, self_ask_prompt, gold_all, ASPECTS, stop=3)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Doc 0 (id=E09-1056) | #sentences=44
--------------------------------------------------------------------------------
lenght= 1600


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


  -> Challenge
     Predicted: [2, 6, 11, 13, 14, 37]
     Gold:      [1, 2, 7, 11]
     Metrics: P=0.33, R=0.50, F1=0.40

lenght= 1602


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


...

  -> Outcome  
     Predicted: [5, 6, 20, 21, 22, 26]
     Gold:      [5, 6, 13, 20, 21, 22, 26]
     Metrics: P=1.00, R=0.86, F1=0.92

Doc 2 (id=P01-1040) | #sentences=17
--------------------------------------------------------------------------------
lenght= 1031


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


  -> Challenge
     Predicted: [1, 5, 6]
     Gold:      [1, 5, 6]
     Metrics: P=1.00, R=1.00, F1=1.00

lenght= 1033


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


  -> Approach 
     Predicted: [2, 7, 8, 12, 14]
     Gold:      [2, 7, 14]
     Metrics: P=0.60, R=1.00, F1=0.75

lenght= 1031
  -> Outcome  
     Predicted: [9, 12, 17]
     Gold:      [3, 4, 12, 13]
     Metrics: P=0.33, R=0.25, F1=0.29


--- Final Aggregate Metrics ---
  -> Challenge Average:
     Precision: 0.64
     Recall:    0.83
     F1-score:  0.72

  -> Approach Average:
     Precision: 0.51
     Recall:    0.93
     F1-score:  0.64

  -> Outcome Average:
     Precision: 0.56
     Recall:    0.54
     F1-score:  0.54

